In [ ]:
import tensorflow as tf
import numpy as np
import sklearn.datasets
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

In [ ]:
def get_nonlinear_data(size=3000, kind='circles', factor=0.3, noise=0.2):
    if kind == 'circles':
        return sklearn.datasets.make_circles(n_samples=size, factor=factor, noise=noise)
    
    if kind == 'moons':
        return sklearn.datasets.make_moons(n_samples=size, noise=noise)
    
    raise ValueError(f"unknown kind {kind}")


In [ ]:
X, labels = get_nonlinear_data(kind='moons', noise=0.2)
plt.scatter(X[:,0], X[:, 1], c=labels, alpha=0.3)

In [ ]:
def get_model(activation='relu', hidden_units=[10, 10], 
              input_shape=(None, 2), output_shape=(None, 2)):
    
    activation_map = {'sigmoid': tf.sigmoid, 'relu': tf.nn.relu}
    
    input_X = tf.placeholder(shape=input_shape, name='input_X', dtype=tf.float32)
    output_Y = tf.placeholder(shape=output_shape, name='output_X', dtype=tf.float32)
    curr_layer = input_X
    curr_size = input_shape[1]
    for l, size in enumerate(hidden_units):
        hidden_layer_W = tf.get_variable(name=f'HiddenLayerW{l}', 
                                         shape=(curr_size, size), 
                                         initializer=tf.random_normal_initializer(seed=0))
        
        hidden_layer_b = tf.get_variable(name=f'HiddenLayerB{l}', 
                                         shape=(1, size), 
                                         initializer=tf.random_normal_initializer(seed=0))
        
        curr_layer = activation_map[activation](tf.matmul(curr_layer, hidden_layer_W) + hidden_layer_b)
        curr_size = size
    
    softmax_layer_W = tf.get_variable(name='SoftmaxLayerW', shape=(curr_size, output_shape[1]),
                                     initializer=tf.random_normal_initializer(seed=0))
    
    softmax_layer_b = tf.get_variable(name='SoftmaxLayerB', shape=(1, output_shape[1]),
                                     initializer=tf.random_normal_initializer(seed=0))
    
    curr_layer = tf.nn.softmax(tf.matmul(curr_layer, softmax_layer_W) + softmax_layer_b)
    
    return input_X, output_Y, curr_layer
        

In [ ]:
input_X, output_Y, curr_layer = get_model()

In [ ]:
loss = -tf.reduce_mean(tf.reduce_mean(output_Y*tf.log(curr_layer), reduction_indices=1))

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

In [ ]:
def batchify(x, y, size=100):
    counter = 0
    while counter < x.shape[0]:
        yield x[counter:counter+size, :], y[counter:counter+size, :]
        counter += size

In [ ]:
Y = OneHotEncoder().fit_transform(labels.reshape(len(labels), 1)).toarray()

EPOCH = 1000
BATHC_SIZE = 100
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(EPOCH):
        for x_batch, y_batch in batchify(X, Y):
            loss_val, _ = sess.run([loss, optimizer], feed_dict={input_X: x_batch, output_Y: y_batch})
        if epoch % 100 == 0:
            print("epoch {} batch loss {}".format(epoch, loss_val), end='')
            loss_val, class_probs = sess.run([loss, curr_layer], feed_dict={input_X: X, output_Y: Y})
            pred_label = np.argmax(class_probs, axis=1)
            acc = accuracy_score(labels, pred_label)
            print(" total loss {} accuray {}".format(loss_val, acc))
    print("last batch loss {}".format(loss_val), end='')
    loss_val, class_probs = sess.run([loss, curr_layer], feed_dict={input_X: X, output_Y: Y})
    pred_label = np.argmax(class_probs, axis=1)
    acc = accuracy_score(labels, pred_label)
    print(" total loss {} accuray {}".format(loss_val, acc))

In [ ]:
probs = np.squeeze(class_probs[:, 1])
plt.scatter(X[:, 0], X[:, 1], c=probs, alpha=0.3)